In [78]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import optuna
from sklearn.model_selection import cross_val_score

In [62]:
import mlflow
mlflow.set_tracking_uri("http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/")

In [63]:
mlflow.set_experiment("Exp 9 - lightGBM HPT on Custom Features")

<Experiment: artifact_location='s3://ms-mlflow-bucket/334855376457357198', creation_time=1751119394395, experiment_id='334855376457357198', last_update_time=1751119394395, lifecycle_stage='active', name='Exp 9 - lightGBM HPT on Custom Features', tags={}>

In [64]:
# Load dataset
dataset = pd.read_csv('data_preprocessed.csv')

# Drop rows with NaN values in 'clean_comment'
cleaned_dataset = dataset.dropna().drop_duplicates()

In [65]:
# Separate features and target
X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)


In [66]:
# Load spacy language model for POS tagging
nlp = spacy.load('en_core_web_sm')

In [67]:
# All POS tags we care about
ALL_POS_TAGS = ['NOUN', 'VERB', 'ADJ', 'ADV', 'PRON', 'PROPN', 'INTJ', 'NUM', 'DET', 'ADP', 'CCONJ', 'PART', 'SCONJ', 'SYM', 'X', 'PUNCT', 'SPACE']

def extract_custom_features(text):
    doc = nlp(text)
    word_list = [token.text for token in doc]

    comment_length = len(text)
    word_count = len(word_list)
    avg_word_length = sum(len(word) for word in word_list) / word_count if word_count > 0 else 0
    unique_word_count = len(set(word_list))
    lexical_diversity = unique_word_count / word_count if word_count > 0 else 0

    pos_tags = [token.pos_ for token in doc]
    pos_proportion = {tag: pos_tags.count(tag) / word_count for tag in ALL_POS_TAGS}
    
    return {
        'comment_length': comment_length,
        'word_count': word_count,
        'avg_word_length': avg_word_length,
        'unique_word_count': unique_word_count,
        'lexical_diversity': lexical_diversity,
        **pos_proportion
    }


In [68]:
# Apply the custom feature extraction
train_custom_features = pd.DataFrame([extract_custom_features(text) for text in X_train_cleaned])
test_custom_features = pd.DataFrame([extract_custom_features(text) for text in X_test_cleaned])

In [69]:
train_custom_features.shape, test_custom_features.shape

((28994, 22), (7249, 22))

In [70]:
# Replace NaN values in POS tag proportions with 0
train_custom_features.fillna(0, inplace=True)
test_custom_features.fillna(0, inplace=True)

In [71]:
# Apply TfidfVectorizer with trigram setting and max_features=1000
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=2000)
X_train_tfidf = tfidf.fit_transform(X_train_cleaned)
X_test_tfidf = tfidf.transform(X_test_cleaned)

In [72]:
# Convert TF-IDF to DataFrame
X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
X_test_tfidf_df = pd.DataFrame(X_test_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [73]:
# Combine TF-IDF and custom features
X_train_combined = pd.concat([X_train_tfidf_df.reset_index(drop=True), train_custom_features.reset_index(drop=True)], axis=1)
X_test_combined = pd.concat([X_test_tfidf_df.reset_index(drop=True), test_custom_features.reset_index(drop=True)], axis=1)

In [74]:
X_train_combined.shape, X_test_combined.shape

((28994, 2022), (7249, 2022))

In [79]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run():
        # Log model type and trial number
        mlflow.set_tag("mlflow.runName", f"Trial_{trial_number}_{model_name}_class_weight_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Log hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy =  cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

        return accuracy




In [80]:
# Function to optimize LightGBM hyperparameters
def objective(trial):
    # Define hyperparameters to be tuned
    param = {
        "objective": "multiclass",
        "num_class": 3,  # Assuming 3 categories (-1, 0, 1)
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "metric": "multi_logloss",
        "is_unbalance": True,
        "class_weight": "balanced",
    }

    # Define the LightGBM model with the trial parameters
    model = lgb.LGBMClassifier(**param)

    # Perform cross-validation
    accuracy = log_mlflow("LightGBM", model, X_train_combined, X_test_combined, y_train_cleaned, y_test_cleaned, param, trial.number)

    # Return the average score across folds
    return accuracy

In [81]:
# Create an Optuna study to optimize the hyperparameters
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2025-06-28 19:46:58,510] A new study created in memory with name: no-name-cb8a593b-430e-4c6f-9f90-364c97dd8796


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.115705 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 107438
[LightGBM] [Info] Number of data points in the train set: 28994, number of used features: 1954
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

2025/06/28 19:49:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 19:49:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_0_LightGBM_class_weight_TFIDF_Trigrams at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/334855376457357198/runs/37c1d69926884203a7863fdc013d84e0
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/334855376457357198


[I 2025-06-28 19:49:36,046] Trial 0 finished with value: 0.807511703462232 and parameters: {'learning_rate': 0.029870459633479507, 'n_estimators': 461, 'max_depth': 11}. Best is trial 0 with value: 0.807511703462232.


...

[I 2025-06-28 21:35:37,504] Trial 48 finished with value: 0.8260329445645459 and parameters: {'learning_rate': 0.07369240870342915, 'n_estimators': 301, 'max_depth': 20}. Best is trial 39 with value: 0.8271711003781195.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.109725 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 107438
[LightGBM] [Info] Number of data points in the train set: 28994, number of used features: 1954
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

2025/06/28 21:37:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 21:38:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_49_LightGBM_class_weight_TFIDF_Trigrams at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/334855376457357198/runs/3b65f94b62e140ba80e607067870503c
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/334855376457357198


[I 2025-06-28 21:38:10,308] Trial 49 finished with value: 0.824756791105797 and parameters: {'learning_rate': 0.06519370397632611, 'n_estimators': 464, 'max_depth': 16}. Best is trial 39 with value: 0.8271711003781195.


In [83]:
model = lgb.LGBMClassifier(

    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance= True,
    class_weight= "balanced",
    reg_alpha= 0.1,  # L1 regularization
    reg_lambda= 0.1,  # L2 regularization
    learning_rate= 0.06631684007924099,
    max_depth= 18,
    n_estimators=449
)

In [84]:
# Fit the model on the resampled training data
model.fit(X_train_combined, y_train_cleaned)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.096177 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 107438
[LightGBM] [Info] Number of data points in the train set: 28994, number of used features: 1954
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

LGBMClassifier(class_weight='balanced', is_unbalance=True,
               learning_rate=0.06631684007924099, max_depth=18,
               metric='multi_logloss', n_estimators=449, num_class=3,
               objective='multiclass', reg_alpha=0.1, reg_lambda=0.1)

In [85]:
# Predict on the test set
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test_combined)
accuracy = accuracy_score(y_test_cleaned, y_pred)
accuracy

0.8391502276176024

In [86]:
from sklearn.metrics import classification_report
# Generate classification report
report = classification_report(y_test_cleaned, y_pred)
print(report)

              precision    recall  f1-score   support

          -1       0.75      0.74      0.74      1657
           0       0.83      0.94      0.88      2393
           1       0.90      0.81      0.86      3199

    accuracy                           0.84      7249
   macro avg       0.83      0.83      0.83      7249
weighted avg       0.84      0.84      0.84      7249

